In [1]:
import pulp
import numpy as np 
import pandas as pd
import math

In [2]:
df = pd.read_csv('sampleOwnIt.csv')

In [8]:
class OwnItRoomAssigner:
    def __init__(self, df,num_rooms,num_attendees):
        self.df = df
        self.rooms = [i for i in range(1,num_rooms+1)]
        self.attendees = [i for i in range(num_attendees)]
    
    def _pref_value_per_person(self,person_idx,room_idx):
        row = self.df.loc[person_idx]
        for k in range(1,6):
            if room_idx == row["Preference {}".format(k)]:
                if row["Status"] == "Early Bird":
                    return (6-k)/10+1
                else:
                    return 0
                    # return (6-k)/10+1-0.05
        return 0

    def _solve_lp_problem(self):
        self.prob = pulp.LpProblem("Matching_OWNIT_Attendees_To_Rooms", pulp.LpMaximize)
        self.y = pulp.LpVariable.dicts("pair", [(i,j) for i in self.attendees for j in self.rooms],cat='Binary')
        self.prob += pulp.lpSum([self._pref_value_per_person(i,j) * self.y[(i,j)] for i in self.attendees for j in self.rooms])
        # One Person can be in One Room
        for i in self.attendees:
            self.prob += pulp.lpSum(self.y[(i,j)] for j in self.rooms) <= 1
        # Rooms should have a minimum number of attendees
        for j in self.rooms:
            self.prob  += pulp.lpSum(self.y[(i,j)] for i in self.attendees) >= math.floor(len(self.attendees)/len(self.rooms))
        # There should be as many assignments as there are attendees
        self.prob += pulp.lpSum(self.y[(i,j)] for i in self.attendees for j in self.rooms) == len(self.attendees)
        self.prob.solve()
    
    def generate_room_assignments(self):
        self._solve_lp_problem()
        for i in self.attendees:
            for j in self.rooms:
                if self.y[(i,j)].varValue == 1:
                    print('{} is in Room {}'.format(self.df.loc[i,"Name"],j))

In [9]:
ora = OwnItRoomAssigner(df=df,num_rooms=7,num_attendees=8)

In [10]:
df

,Name,Status,Preference 1,Preference 2,Preference 3,Preference 4,Preference 5
0,Mitali,Early Bird,1,2,6,4,5
1,Bibi,Early Bird,1,5,3,4,2
2,Annie,Regular,1,4,3,2,5
3,Shirin,Early Bird,3,2,6,1,5
4,Lindsay,Regular,4,3,1,5,2
5,Shep,Early Bird,5,2,1,6,7
6,Lena,Regular,6,2,3,1,4
7,Ben,Early Bird,1,2,3,4,5


In [7]:
ora.generate_room_assignments()

Mitali is in Room 2
Bibi is in Room 5
Annie is in Room 1
Shirin is in Room 3
Lindsay is in Room 4
Shep is in Room 7
Lena is in Room 6
Ben is in Room 1


In [ ]:
Mitali is in Room 2
Bibi is in Room 1
Annie is in Room 7
Shirin is in Room 3
Lindsay is in Room 6
Shep is in Room 5
Lena is in Room 4
Ben is in Room 1